In [ ]:
%pdb on

In [ ]:
import sys
import random
import math
import operator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
import os.path
from IPython.core.debugger import Tracer
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
a=np.zeros([3,2])

In [ ]:
a[0,:]=[5,4]

In [ ]:
a

In [ ]:
campaign="2997"
folder='../../make-ipinyou-data'
resultsfolder='../results'
trainYzxFile=os.path.join(folder,campaign,'train.yzx.txt')
testYzxFile=os.path.join(folder,campaign,'test.yzx.txt')

In [ ]:
bufferCaseNum = 1000000
#bufferCaseNum = 200
eta = 0.01 # learning rate
lamb = 1E-6 # regularization parameter
featWeight = {}
trainRounds = 10
#trainRounds = 1
random.seed(10)
initWeight = 0.05

def nextInitWeight():
    return (random.random() - 0.5) * initWeight
def ints(s):
    res = []
    for ss in s:
        res.append(int(ss))
    return res
def sigmoid(p):
    return 1.0 / (1.0 + math.exp(-p))

In [ ]:
for round in range(0, trainRounds):
    # train for this round
    fi = open(trainYzxFile, 'r')
    lineNum = 0
    trainData = []
    y = []
    yp = []
    for line in fi:
        lineNum = (lineNum + 1) % bufferCaseNum
        trainData.append(ints(line.replace(":1", "").split()))
        if lineNum == 0:
            for data in trainData:
                clk = data[0]
                mp = data[1]
                fsid = 2 # feature start id
                # predict
                pred = 0.0
                for i in range(fsid, len(data)):
                    feat = data[i]
                    if feat not in featWeight:
                        featWeight[feat] = nextInitWeight()
                    pred += featWeight[feat]
                pred = sigmoid(pred)
                y.append(clk)
                yp.append(pred)
                # start to update weight
                # w_i = w_i + learning_rate * [ (y - p) * x_i - lamb * w_i ] 
                for i in range(fsid, len(data)):
                    feat = data[i]
                    featWeight[feat] = featWeight[feat] * (1 - lamb) + eta * (clk - pred)
            trainData = []
    # complete iterations when nr lines < bufferCaseNum
    if len(trainData) > 0:
        for data in trainData:
            clk = data[0] # click
            mp = data[1] # market price
            fsid = 2 # feature start id
            # predict
            pred = 0.0
            for i in range(fsid, len(data)):
                feat = data[i]
                if feat not in featWeight:
                    featWeight[feat] = nextInitWeight()
                pred += featWeight[feat]
            pred = sigmoid(pred)
            y.append(clk)
            yp.append(pred)
            # start to update weight
            # w_i = w_i + learning_rate * [ (y - p) * x_i - lamb * w_i ]
            for i in range(fsid, len(data)):
                feat = data[i]
                featWeight[feat] = featWeight[feat] * (1 - lamb) + eta * (clk - pred)
    fi.close()
    auc = roc_auc_score(y, yp)
    rmse = math.sqrt(mean_squared_error(y, yp))
    print 'Train: ' + '\t' + str(round) + '\t' + str(auc) + '\t' + str(rmse)
    
    # test for this round
    y = []
    yp = []
    fi = open(testYzxFile, 'r')
    for line in fi:
        data = ints(line.replace(":1", "").split())
        clk = data[0] # click
        mp = data[1] # market price
        fsid = 2 # feature start id
        pred = 0.0
        for i in range(fsid, len(data)):
            feat = data[i]
            if feat in featWeight:
                pred += featWeight[feat]
        pred = sigmoid(pred)
        y.append(clk)
        yp.append(pred)
    fi.close()
    auc = roc_auc_score(y, yp)
    rmse = math.sqrt(mean_squared_error(y, yp))
    print 'Test: ' + '\t' + str(round) + '\t' + str(auc) + '\t' + str(rmse)
    Tracer()()
    
# output the weights
fo = open(trainYzxFile + '.lr.weight', 'w')
featvalue = sorted(featWeight.iteritems(), key=operator.itemgetter(0))
for fv in featvalue:
    fo.write(str(fv[0]) + '\t' + str(fv[1]) + '\n')
fo.close()


# output the prediction
fi = open(testYzxFile, 'r')
fo = open(testYzxFile + '.lr.pred', 'w')

for line in fi:
    data = ints(line.replace(":1", "").split())
    pred = 0.0
    for i in range(1, len(data)):
        feat = data[i]
        if feat in featWeight:
            pred += featWeight[feat]
    pred = sigmoid(pred)
    fo.write(str(pred) + '\n')    
fo.close()
fi.close()





eta = 0.01 # learning rate
lamb = 1E-6 # regularization parameter
0	0.602293015776	0.0581610175761
1	0.605309575144	0.0581604886761
2	0.60586774705	0.0581606929815
3	0.606052831839	0.0581612808149
4	0.60611824294	0.0581622062518
5	0.605952622564	0.0581634308069
6	0.605726667968	0.0581649071177
7	0.605290935649	0.0581665820742
8	0.604877688397	0.0581684017922
9	0.604405592989	0.0581703155207

In [ ]:
len(clkPred)

In [ ]:
type(clkPred)

In [ ]:
clkTime=clkPred[:,0]
predTime=clkPred[:,1]
plt.plot(clkTime)
plt.plot(predTime)
yRange=clkPred.max() - clkPred.min()
plt.ylim(clkPred.min() - 0.1*yRange, clkPred.max() + 0.1*yRange)

In [ ]:
fi = open(trainYzxFile, 'r')
type(fi)

In [ ]:
help("file")